In [1]:
import os

import pandas as pd

In [2]:
def extract_columns(model_result_fp, columns):
    all_dfs = []
    datasets = None
    for model, result_fp in model_result_fp.items():
        results_dataframe = pd.read_csv(result_fp, sep='\t')
        if datasets is None:
            datasets = results_dataframe['dataset']
        results_dataframe = results_dataframe.set_index('dataset')
        results_dataframe = results_dataframe[columns]
        new_col_names = {column : '{} {}'.format(model, column) for column in columns}
        results_dataframe = results_dataframe.rename(columns=new_col_names)

        all_dfs.append(results_dataframe)
    return pd.concat(all_dfs, axis=1)

def columns_avliable(model_result_fp):
    for model, result_fp in model_result_fp.items():
        results_dataframe = pd.read_csv(result_fp, sep='\t')
        return [col for col in results_dataframe.columns if 'Unnamed' not in col]
    

def add_mean(results_df, rounding=2):
    mean_results = results_df.mean()
    mean_results.name = 'Mean'
    results_df = results_df.append(mean_results).round(rounding)
    return results_df
    

# Mass Evaluation Result analysis

Below we are matching the name of the model to the result file for that model

In [7]:
root_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
root_folder_join = lambda paths: os.path.join(root_folder, 'results', *paths)

target_dep_results_fp = root_folder_join(['Target Dependent', 'Target Dependent.tsv'])
target_dep_plus_results_fp = root_folder_join(['Target Dependent', 'Target Dependent+.tsv'])
tdparse_results_fp = root_folder_join(['TDParse','TDParse.tsv'])
tdparse_plus_results_fp = root_folder_join(['TDParse','TDParsePlus.tsv'])
lstm_results_fp = root_folder_join(['TDLstm', 'lstm', 'results file.tsv'])
tdlstm_results_fp = root_folder_join(['TDLstm', 'tdlstm', 'results file.tsv'])
tclstm_results_fp = root_folder_join(['TDLstm', 'tclstm', 'results file.tsv'])

model_result_fp = {'Target-Dep' : target_dep_results_fp, 'Target-Dep+' : target_dep_plus_results_fp,
                   'TDParse' : tdparse_results_fp, 'TDParse+' : tdparse_plus_results_fp,
                   'LSTM' : lstm_results_fp, 'TDLSTM' : tdlstm_results_fp, 'TCLSTM' : tclstm_results_fp}

Below we extract out the 3 way Macro F1 scores for each method on all of the dataset and combine them together into one table.

In [16]:
f1_results = extract_columns(model_result_fp, ['3 Class Macro F1'])
f1_results = add_mean(f1_results)
f1_results

,Target-Dep 3 Class Macro F1,Target-Dep+ 3 Class Macro F1,TDParse 3 Class Macro F1,TDParse+ 3 Class Macro F1,LSTM 3 Class Macro F1,TDLSTM 3 Class Macro F1,TCLSTM 3 Class Macro F1
Dong Twitter,65.70,65.70,66.00,68.10,62.00,63.00,60.10
Election Twitter,45.50,45.90,46.20,44.60,39.20,43.10,41.90
Mitchel,40.80,42.90,40.50,50.00,39.90,41.60,35.50
SemEval 14 Laptop,60.00,63.70,59.60,64.50,58.20,56.10,47.30
SemEval 14 Restaurant,56.20,57.70,59.40,61.00,48.70,59.80,49.60
YouTuBean,53.10,55.60,71.70,68.00,45.20,43.90,34.70
Mean,53.55,55.25,57.23,59.37,48.87,51.25,44.85


We do the same as above but for the Accuracy

In [10]:
acc_results = extract_columns(model_result_fp, ['Accuracy'])
acc_results = add_mean(acc_results)
acc_results

,Target-Dep Accuracy,Target-Dep+ Accuracy,TDParse Accuracy,TDParse+ Accuracy,LSTM Accuracy,TDLSTM Accuracy,TCLSTM Accuracy
Dong Twitter,67.30,67.80,68.60,70.10,65.30,65.60,63.70
Election Twitter,57.70,56.60,57.10,56.40,52.00,55.30,56.20
Mitchel,72.60,72.80,72.90,74.50,70.70,70.80,70.10
SemEval 14 Laptop,67.90,70.80,68.00,71.30,66.50,63.80,56.10
SemEval 14 Restaurant,73.80,74.60,73.30,76.20,73.30,74.50,68.40
YouTuBean,70.80,72.50,81.20,80.80,72.50,71.70,65.40
Mean,68.35,69.18,70.18,71.55,66.72,66.95,63.32


We can then look at the 2 way Macro F1 score which evaluates how well the models can predict Positive or Negative sentiment while still taking into account prediciting the neutral class

In [11]:
two_way_f1_results = extract_columns(model_result_fp, ['2 Class F1'])
two_way_f1_results = add_mean(two_way_f1_results)
two_way_f1_results

,Target-Dep 2 Class F1,Target-Dep+ 2 Class F1,TDParse 2 Class F1,TDParse+ 2 Class F1,LSTM 2 Class F1,TDLSTM 2 Class F1,TCLSTM 2 Class F1
Dong Twitter,62.40,61.90,61.50,64.40,57.30,58.30,54.3
Election Twitter,40.00,41.00,41.20,38.90,34.70,36.30,35.8
Mitchel,19.20,22.50,18.80,32.70,18.40,21.20,12.2
SemEval 14 Laptop,68.60,71.80,69.30,72.00,68.20,64.80,56.2
SemEval 14 Restaurant,73.60,73.80,70.90,76.40,73.10,69.20,59.4
YouTuBean,39.00,42.30,64.00,58.50,25.90,24.30,13.1
Mean,50.47,52.22,54.28,57.15,46.27,45.68,38.5


Now we look at the results of trying to predict the sentiment of all targets that are only in sentence that contain one distinct sentiment. This should be a result that is good for the LSTM model as it does not take into account any target information. However as we can see below the LSTM is one of the worse models.

In [12]:
one_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 1 distinct sentiments'])
one_dist_sent = add_mean(one_dist_sent)
one_dist_sent

,Target-Dep 3 Class Macro F1 for text with 1 distinct sentiments,Target-Dep+ 3 Class Macro F1 for text with 1 distinct sentiments,TDParse 3 Class Macro F1 for text with 1 distinct sentiments,TDParse+ 3 Class Macro F1 for text with 1 distinct sentiments,LSTM 3 Class Macro F1 for text with 1 distinct sentiments,TDLSTM 3 Class Macro F1 for text with 1 distinct sentiments,TCLSTM 3 Class Macro F1 for text with 1 distinct sentiments
Dong Twitter,65.70,65.7,66.00,68.1,62.00,63.00,60.10
Election Twitter,50.30,50.3,48.70,48.6,46.50,46.00,41.70
Mitchel,41.50,43.1,41.30,51.3,40.90,41.80,36.20
SemEval 14 Laptop,61.00,67.2,60.70,67.3,61.60,57.70,47.20
SemEval 14 Restaurant,57.20,60.5,57.70,62.7,51.70,58.70,47.10
YouTuBean,54.20,57.0,73.40,69.2,46.00,44.60,35.20
Mean,54.98,57.3,57.97,61.2,51.45,51.97,44.58


We do the same as above but for 2 distinct sentiments per sentence. We remove the Dong twitter dataset from this experiment due to it only containing Tweets with one distinct sentiment.

In [13]:
two_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 2 distinct sentiments'])
two_dist_sent = two_dist_sent.drop('Dong Twitter')
two_dist_sent = add_mean(two_dist_sent)
two_dist_sent

,Target-Dep 3 Class Macro F1 for text with 2 distinct sentiments,Target-Dep+ 3 Class Macro F1 for text with 2 distinct sentiments,TDParse 3 Class Macro F1 for text with 2 distinct sentiments,TDParse+ 3 Class Macro F1 for text with 2 distinct sentiments,LSTM 3 Class Macro F1 for text with 2 distinct sentiments,TDLSTM 3 Class Macro F1 for text with 2 distinct sentiments,TCLSTM 3 Class Macro F1 for text with 2 distinct sentiments
Election Twitter,41.90,43.00,43.90,41.50,33.70,40.30,41.80
Mitchel,31.60,34.80,30.60,37.90,27.40,35.60,28.00
SemEval 14 Laptop,51.90,47.40,50.70,49.70,39.60,46.90,43.80
SemEval 14 Restaurant,48.50,48.60,58.00,53.80,37.00,55.80,50.20
YouTuBean,31.50,31.10,42.20,50.90,20.50,20.50,20.50
Mean,41.08,40.98,45.08,46.76,31.64,39.82,36.86


Lastly the same as above but for 3 distinct sentiments. As the majority of the datasets have very few 3 distinct sentiments per sentence we only look at the Twitter Elections datases (8.78% of the dataset contains 3 distinct sentiments)

In [14]:
three_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 3 distinct sentiments'])
three_dist_sent = pd.DataFrame(three_dist_sent.loc['Election Twitter'])
three_dist_sent.sort_values('Election Twitter', ascending=False)

,Election Twitter
TDParse 3 Class Macro F1 for text with 3 distinct sentiments,44.9
TDLSTM 3 Class Macro F1 for text with 3 distinct sentiments,43.3
TCLSTM 3 Class Macro F1 for text with 3 distinct sentiments,42.7
TDParse+ 3 Class Macro F1 for text with 3 distinct sentiments,42.0
Target-Dep 3 Class Macro F1 for text with 3 distinct sentiments,41.9
Target-Dep+ 3 Class Macro F1 for text with 3 distinct sentiments,40.9
LSTM 3 Class Macro F1 for text with 3 distinct sentiments,29.5
